In [30]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
from torch import nn, optim
from collections import OrderedDict
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
"""处理数据，构建数据集部分"""

'处理数据，构建数据集部分'

In [67]:
# 使用协同过滤算法生成训练样本, 使用ItemCF
def load_data(path):
    Data = pd.read_csv(path, nrows=10000)
    Ratings = Data.pivot_table(index='userId', columns='movieId', values='rating')
    return Ratings
UserItemMatrix = load_data(r'G:\github项目\Recomendation_system\Data\movie\ratings.csv')

In [68]:
UserItemMatrix

movieId,1,2,3,4,5,6,7,8,10,11,...,182715,183611,184471,185031,185135,187541,187593,187595,188301,190183
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,NaN,4.0,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,...,4.0,4.0,3.5,4.5,5.0,NaN,4.0,4.0,NaN,NaN
63,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,4.0,NaN,3.5,NaN,NaN,4.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
def person_similarity(UserItemMatrix, based='user'):
    """计算皮尔逊相关系数，这里是基于用户的UserCF，若计算ItemCF可改为item"""
    if based == 'user':
        similarity = UserItemMatrix.T.corr(method='pearson') # corr是按列计算相关性，故这里计算用户相关性需要将用户矩阵转置
    else:
        similarity = UserItemMatrix.corr(method='pearson')
    return similarity
ItemSimilarity = person_similarity(UserItemMatrix, based='item')

In [70]:
def select_similar_item(ItemSimilarity, k, iid):
    ItemSimilarity = ItemSimilarity.loc[iid].dropna()
    SimilarItem = []
    for IId in ItemSimilarity.index:
        if ItemSimilarity.loc[IId] != None:
            SimilarItem.append([IId, ItemSimilarity.loc[IId]])
    SimilarItem.sort(key=lambda x: -x[1])
    SimilarItem = [SimilarItem[i][0] for i in range(len(SimilarItem))]
    return SimilarItem[:k]
select_similar_item(ItemSimilarity, 10, 3)

[5, 3, 15, 24, 54, 62, 86, 87, 92, 93]

In [71]:
def looked_user(UserItemMatrix, iid):
    looked_users = []
    users = UserItemMatrix.loc[:,iid].dropna()
    for index in users.index:
        looked_users.append([index, users.loc[index]])
    looked_users.sort(key=lambda x: -x[1])
    looked_users = [looked_users[i][0] for i in range(len(looked_users))]
    return looked_users
looked_user(UserItemMatrix, 1)

[31,
 40,
 43,
 46,
 57,
 63,
 7,
 17,
 1,
 5,
 19,
 45,
 64,
 66,
 18,
 21,
 27,
 32,
 33,
 44,
 50,
 54,
 15]

In [72]:
def predict_score(uid, iid, UserItemMatrix, similarity):
    similar_item = similarity.loc[iid].drop([iid]).dropna()
    similar_item = similar_item.where(similar_item >= 0.1).dropna()
    indexes = set(UserItemMatrix.loc[uid].dropna().index) & set(similar_item.index) # 筛选出对相似物品有过评分的用户
    similar_item = similar_item.loc[list(indexes)]

    sum_up = 0
    sum_down = 0
    for sim_iid, similarity in similar_item.items():
        sim_item_similarity_user = UserItemMatrix.loc[:,sim_iid]
        sim_item_rating_for_user = sim_item_similarity_user.loc[uid]
        sum_up += similarity * sim_item_rating_for_user
        sum_down += similarity
    return sum_up / (sum_down + 1e-5)
predict_score(1, 1, UserItemMatrix, ItemSimilarity)  # 预测uid为1对iid为1的评分，实际为4

4.56304053705267

In [ ]:
def predict_all(UserItemMatrix, similarity):
    """预测全部用户对物品的评分"""
    user_id = UserItemMatrix.index
    item_id = UserItemMatrix.columns
    Predict_Matrix = pd.DataFrame(index=user_id, columns=item_id) # 新建一个与原先用户评分矩阵相同的空DataFrame
    error = 0
    actual_value = 0
    for uid in user_id:
        for iid in item_id:
            Predict_Matrix.loc[uid].loc[iid] = predict_score(uid, iid, UserItemMatrix, similarity)
            if UserItemMatrix.loc[uid].loc[iid] != None:
                error += abs(UserItemMatrix.loc[uid].loc[iid] - Predict_Matrix.loc[uid].loc[iid])  # 按照已有的评分进行准确率评估
                actual_value += UserItemMatrix.loc[uid].loc[iid]
    accuracy = error / actual_value
    return Predict_Matrix, accuracy
Pre, acc = predict_all(UserItemMatrix, ItemSimilarity)

In [ ]:
Pre

In [11]:
ind = UserItemMatrix.index
col = UserItemMatrix.columns
Pre = pd.DataFrame(Pre, index=ind, columns=col)

In [12]:
def add_zeros(UserItemMatrix, Pre):
    # 那些0值用每一个物品的平均评分来代替
    ind = UserItemMatrix.index
    col = UserItemMatrix.columns
    average = []
    for iid in UserItemMatrix.columns:
        average.append(sum(Pre.loc[:,iid]) / len(Pre.loc[:,iid]))
    for uid in Pre.index:
        for i in range(len(Pre.columns)):
            if Pre.loc[uid].iloc[i] == 0:
                Pre.loc[uid].iloc[i] = average[i]
    return Pre
Pre = add_zeros(UserItemMatrix, Pre)

In [13]:
Pre

movieId,1,2,3,4,5,6,7,8,10,11,...,91529,91658,99114,106782,109487,112552,114060,115713,122882,131724
userId,,,,,,,,,,,,,,,,,,,,,
1,3.333328,0.0,3.555552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.69047,0.0,2.412694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.69047,0.0,2.412694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.999987,0.0,2.412694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.499988,0.0,3.499991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.499993,0.0,4.999994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.499994,0.0,4.833317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
def split_data(Data, ratio=0.8):
    Train_Data = Data.loc[:,:len(Data.columns) * ratio].T
    Val_Data = Data.loc[:, len(Data.columns)*ratio:].T
    Train_Data = Train_Data.values
    Val_Data = Val_Data.values
    Train_Data = torch.from_numpy(Train_Data.astype(float))
    Val_Data = torch.from_numpy(Val_Data.astype(float))
    return Train_Data, Val_Data 
Train_Data, Val_Data = split_data(Pre)

In [15]:
Train_Data

tensor([[3.3333, 2.6905, 2.6905,  ..., 2.5000, 4.5000, 4.5000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [3.5556, 2.4127, 2.4127,  ..., 3.5000, 5.0000, 4.8333],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [16]:
num_feature = len(Train_Data[1])

In [63]:
Train_Data.shape[1]

7

In [18]:
class MyDataset(Dataset):
    def __init__(self, Data):
        self.Data = Data
        self.x = Data
        self.y = Data
        
    def __getitem__(self, index):
        sample = {'x':self.x[index], 'y':self.y[index]}
        return sample
    
    def __len__(self):
        return len(self.Data)

In [19]:
train_set = MyDataset(Train_Data)
val_set = MyDataset(Val_Data)

In [20]:
"""搭建模型部分"""

'搭建模型部分'

In [60]:
class AutoRec(nn.Module):
    def __init__(self, num_users, num_items, hidden_units):
        super(AutoRec, self).__init__()
        
        self.num_users = num_users
        self.num_items = num_items
        self.hidden_units = hidden_units
        
        self.enc_linear1 = nn.Linear(self.num_items, self.hidden_units)
        self.enc_relu1 = nn.ReLU()
        
        self.dec_linear1 = nn.Linear(self.hidden_units, self.num_items)
        
    def forward(self, x):
        x = self.enc_relu1(self.enc_linear1(x))
        x = self.dec_linear1(x)
        return x

In [61]:
"""模型训练部分"""

'模型训练部分'

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batchsize = 1  # 每一批的数据数量

# 定义模型
num_users = Train_Data.shape[0]
num_items = Train_Data.shape[1]
model = AutoRec(num_users, num_items, 300)
# AutoRec = AutoRec.to(device)

# 定义损失函数
loss_fn = nn.MSELoss()
# loss_fn = loss_fn.to(device)

# 定义优化器
lr = 0.01  # 学习率
optimal = optim.SGD(model.parameters(), lr)

# 创建数据迭代集
Train_loader = DataLoader(train_set, batch_size=batchsize)
Val_loader = DataLoader(val_set, batch_size=batchsize)

def train(model, Train_loader, Val_loader, optimizer, loss_fn, lr, batchsize, epoches=100):
    model.train()
    for epoch in range(epoches):
        print("第{}轮训练开始，共{}轮".format(epoch+1, epoches))
        train_loss = 0
        for i, sample in enumerate(Train_loader):
            x = Variable(sample['x'].type(torch.FloatTensor))
            y = Variable(sample['y'])
            output = model(x)
            loss = loss_fn(output, y.float())
            
            # 优化器优化模型
            optimal.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
    
        # 开始验证模式
        model.eval()
        eval_loss = 0
        with torch.no_grad():
            for i, sample in enumerate(Val_loader):
                x_val = Variable(sample['x'].type(torch.FloatTensor))
                y_val = Variable(sample['y'])

                output = model(x_val)
                loss = loss_fn(output, y_val.float())
                eval_loss += loss
        print('第{}轮训练完成， 训练损失为{}， 验证损失为{}'.format(epoch+1, train_loss, eval_loss))
    return model


In [66]:
train(model, Train_loader, Val_loader, optimal, loss_fn, lr, batchsize)

第1轮训练开始，共100轮
第1轮训练完成， 训练损失为76.19556359783019， 验证损失为21.67206573486328
第2轮训练开始，共100轮
第2轮训练完成， 训练损失为11.350853239643584， 验证损失为9.11535358428955
第3轮训练开始，共100轮
第3轮训练完成， 训练损失为5.477324312209021， 验证损失为6.4077019691467285
第4轮训练开始，共100轮
第4轮训练完成， 训练损失为3.1014055690595796， 验证损失为5.125322341918945
第5轮训练开始，共100轮
第5轮训练完成， 训练损失为1.9490946867445018， 验证损失为4.329331874847412
第6轮训练开始，共100轮
第6轮训练完成， 训练损失为1.3442712293859813， 验证损失为3.764054298400879
第7轮训练开始，共100轮
第7轮训练完成， 训练损失为1.0069971084315057， 验证损失为3.3304803371429443
第8轮训练开始，共100轮
第8轮训练完成， 训练损失为0.8024346999548015， 验证损失为2.9807016849517822
第9轮训练开始，共100轮
第9轮训练完成， 训练损失为0.6691497253357852， 验证损失为2.693779945373535
第10轮训练开始，共100轮
第10轮训练完成， 训练损失为0.5749930578824944， 验证损失为2.4522838592529297
第11轮训练开始，共100轮
第11轮训练完成， 训练损失为0.504087172846539， 验证损失为2.244452953338623
第12轮训练开始，共100轮
第12轮训练完成， 训练损失为0.4479147435484663， 验证损失为2.063284158706665
第13轮训练开始，共100轮
第13轮训练完成， 训练损失为0.40146365940353235， 验证损失为1.9033693075180054
第14轮训练开始，共100轮
第14轮训练完成， 训练损失为0.361639452955842， 验证损失为1.76216065883

AutoRec(
  (enc_linear1): Linear(in_features=7, out_features=300, bias=True)
  (enc_relu1): ReLU()
  (dec_linear1): Linear(in_features=300, out_features=7, bias=True)
)